In [8]:
import pandas as pd
import re
from os import listdir
from tqdm import tqdm
from anytree import NodeMixin, RenderTree, PreOrderIter

In [9]:
def toRow(df, entry):
    row = {}
    row['job'] = entry['job']
    row['page'] = df.loc[entry['nodes'][-1]].page.values[0]
    row['file'] = df.loc[entry['nodes'][-1]].filename.values[0]
    levels = ['L1', 'L2', 'L3', 'L4', 'L5', 'L6']
    for i, node in enumerate(entry['nodes']):
        row[levels[i]] = ' '.join(df.loc[node].text.values)
    row['amount'] = df.loc[entry['nodes'][-1][-2]].text
    return row


def get_patern_of_bullet(String):
    regx = [
        ('\d{5}-\d+$', 70),
        ('[1-9][0-9]*(\.[1-9][0-9]*)*\)$', 20),
        ('\(\d*(\.?\d*)*\)$', 50),
        ('[1-9][0-9]*(\.[1-9][0-9]*)+$', 2),
        ('[1-9][0-9]*\.$', 1)
    ]

    for r, l in regx:
        if re.match(r, String):
            if l in [2, 20, 50]:
                l = String.count('.') + l
            return r, l
    return '', 0

In [15]:
df = pd.read_csv('bkk-bud_cleaned.csv', index_col='index')
df = df.sort_values(by=['filename', 'page', 'line_num']).reset_index(drop=True)
fpl_group = df.groupby(['filename', 'page', 'line_num'])

In [16]:
bulletFlag = False
entry = []
entries = []

for i, group in tqdm(fpl_group):
    lineText = group.text.values
    if not i[2]:
        continue

    if (lineText[0].startswith('งาน') and i[2] <= 2) or (lineText[0].startswith('งาน:')):
        entries.append(('job', group.index.to_list()))
        continue

    if ' ' in group.iloc[0].text:
        bullet = get_patern_of_bullet(group.iloc[0].text.split(' ')[0])
    else:
        bullet = get_patern_of_bullet(group.iloc[0].text)

    if lineText[0].startswith('งาน'):
        if (lineText[-1] == 'บาท'):
            lineText = lineText[:-2]
        job = ' '.join(lineText)
        continue
    elif bullet[1]:
        bulletFlag = True

    if bulletFlag:
        entry += group.index.to_list()
        if lineText[-1] == 'บาท':
            bulletFlag = False
            entries.append(('list', entry))
            entry = []


100%|██████████| 43170/43170 [00:15<00:00, 2792.12it/s]


In [17]:
class Budget(NodeMixin):
    def __init__(self, id, name, idx, bullet, parent=None, children=None):
        super(Budget, self).__init__()
        self.name = name
        self.id = id
        self.idx = idx
        self.amount = ''
        if len(idx) > 2:
            self.amount = df.loc[idx[-2]].text
        elif len(name.split(' ')) > 2:
            self.amount = name.split(' ')[-2]

        self.bullet = bullet
        self.parent = parent
        if children:
            self.children = children
    def __repr__(self):
        return self.name


In [18]:
BUDGET_RUNNING_ID = 0
root = Budget(BUDGET_RUNNING_ID, 'BKK2022', [-1], -2)
BUDGET_RUNNING_ID += 1

job_root = None
curr = None

for l, entry in tqdm(entries):
  entryText = ' '.join(df.loc[entry].text.values)

  if l == 'job':
    job_root = Budget(BUDGET_RUNNING_ID, entryText, entry, -1, root)
    BUDGET_RUNNING_ID += 1
    curr = job_root
    continue

  entry_bullet = get_patern_of_bullet(df.loc[entry[0]].text.split(' ')[0])
  if curr == job_root:
     curr = Budget(BUDGET_RUNNING_ID, entryText, entry, entry_bullet[1], job_root)
     BUDGET_RUNNING_ID += 1
  else:
    while curr.bullet != job_root and entry_bullet[1] <= curr.bullet:
      curr = curr.parent
    curr = Budget(BUDGET_RUNNING_ID, entryText, entry, entry_bullet[1], curr)
    BUDGET_RUNNING_ID+=1
  

100%|██████████| 28043/28043 [00:19<00:00, 1424.16it/s]


In [19]:
has_parent = pd.DataFrame(
    [
        {'id': node.id, 'parent': node.parent.id}
        for node in PreOrderIter(root) if node.parent
    ])
item = pd.DataFrame(
    [
        {
            'id': node.id,
            'text': node.name, 
            'amount': node.amount.replace(' ', '').replace(',', ''), 
            'filename': df.loc[node.idx[0]].filename, 
            'page': df.loc[node.idx[0]].page,
            'line_num': df.loc[node.idx[0]].line_num
            }
        for node in PreOrderIter(root)
        if node.is_leaf
    ]
)

has_parent.to_csv('2022bkk-buget-has_parent.csv', index=0)
item.to_csv('2022bkk-buget-item.csv', index=0)